In [85]:
import pandas as pd
import os
from datetime import timedelta


#os.chdir("C:\\Users\\dhung\\OneDrive\\Documents\\asking people for job\\cody\\capstone")
os.chdir('/Users/hungduong/Documents/MSBA UofU/MSBA 4th/IS 6813')

In [86]:
customer = pd.read_csv("Copy of customer.csv")
cutoff = pd.read_csv("Copy of cutoff_times.csv")
ga = pd.read_csv("Copy of google_analytics.csv")
#material  = pd.read_csv("Copy of material.csv")
#hours  = pd.read_csv("Copy of operating_hours.csv")
#orders = pd.read_csv("Copy of orders.csv")
#sales = pd.read_csv("Copy of sales.csv")
visit =  pd.read_csv("visit_plan.csv")
#master = pd.read_csv("master.csv")





# Making of master table from visit and cutoff

In [5]:
visit.head()


,CUSTOMER_ID,FREQUENCY,ELT_TS,SNAPSHOT_DATE,ANCHOR_DATE,SALES_OFFICE,SALES_OFFICE_DESC,DISTRIBUTION_MODE,SHIPPING_CONDITIONS_DESC
0,500405850,02,2021-12-14T05:49:54.551Z,2021-12-13,2020-07-20,G133,"Glenwood Springs, CO",SL,48 Hours
1,500286563,01,2021-12-14T05:49:54.551Z,2021-12-13,2019-11-04,G236,"Denver, CO",OF,48 Hours
2,600057832,01,2021-12-14T05:49:54.551Z,2021-12-13,2021-08-02,G111,"Draper, UT",SL,48 Hours
3,500431853,01,2021-12-14T05:49:54.551Z,2021-12-13,2020-04-06,G111,"Draper, UT",SL,48 Hours
4,600076074,01,2021-12-14T05:49:54.551Z,2021-12-13,2019-06-07,G142,"Pocatello, ID",EZ,24 Hours


In [6]:
cutoff.head(10)

,SALES_OFFICE,PLANT_ID,CUTOFFTIME__C,SHIPPING_CONDITION_TIME,DISTRIBUTION_MODE
0,"Draper, UT",G111,6:00:00 PM,72hrs,Tell Sell
1,"Draper, UT",G111,6:00:00 PM,48hrs,Tell Sell
2,"Draper, UT",G111,1:30:00 PM,24hrs,Tell Sell
3,"Draper, UT",G111,3:30:00 PM,24hrs,OFS
4,"Draper, UT",G111,3:30:00 PM,24hrs,Bulk Distribution
5,"Draper, UT",G111,3:30:00 PM,24hrs,Full Service
6,"Draper, UT",G111,3:30:00 PM,24hrs,Sideload
7,"Logan, UT",G113,1:00:00 PM,24hrs,OFS
8,"Logan, UT",G113,5:00:00 PM,48hrs,OFS
9,"Logan, UT",G113,5:00:00 PM,48hrs,Sideload


In [7]:
# Turn ANCHOR_DATE into date dtype
visit['ANCHOR_DATE'] = pd.to_datetime(visit['ANCHOR_DATE'], errors='coerce')
visit = visit.dropna(subset=['ANCHOR_DATE'])

# Add a day of the week column
visit['WEEK_DAY_OF_ANCHOR_DATE'] = visit['ANCHOR_DATE'].dt.day_name()

# FREQUENCY column, standardize values
clean = visit['FREQUENCY'].astype(str).str.strip()

mapped = clean.replace({
    "01": 1, "1": 1, "1.0": 1, "1 ": 1, " 1": 1, "Every Week On": 1,
    "02": 2, "2": 2, "2.0": 2, "2 ": 2, "Every Second Week On": 2,
    "03": 3, "3": 3, "3.0": 3, "3 ": 3, "Every Third Week On": 3,
    "04": 4, "4": 4, "4.0": 4, "4 ": 4, "Every Fourth Week On": 4,
    "05": 5, "5": 5, "5.0": 5, "5 ": 5, "Every Fifth Week On": 5,
    "06": 6, "6": 6, "6.0": 6, "Every Sixth Week On": 6,
    "08": 8, "8": 8, "8.0": 8, "Every Eighth Week On": 8,
    "10": 10, "10.0": 10, "Every Tenth Week On": 10,
    "Not Applicable": None
})

# Make it into days (week*7)
visit['WINDOW_FREQUENCY'] = (pd.to_numeric(mapped, errors='coerce').astype('Int64')*7)


/var/folders/hd/fbf93pws5pn1ycb22tw5k1240000gn/T/ipykernel_72957/4278182585.py:11: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  mapped = clean.replace({


In [8]:
cutoff['DISTRIBUTION_MODE'].unique()

array(['Tell Sell', 'OFS', 'Bulk Distribution', 'Full Service',
       'Sideload', 'E Pallet', 'Rapid Delivery', 'Night OFS',
       'Nights Bulk', 'Night Rapid Delivery', 'Special Events',
       'Night Sideload'], dtype=object)

In [9]:
visit['DISTRIBUTION_MODE'].unique()

array(['SL', 'OF', 'EZ', 'RD', nan, 'NR', 'FS', 'BK', 'SE', 'NS', 'NO',
       'DD'], dtype=object)

In [10]:
# mapping for code to description
code_to_description = {
    'OF': 'OFS',
    'SL': 'Sideload',
    'BK': 'Bulk Distribution',
    'FS': 'Full Service',
    'RD': 'Rapid Delivery',
    'NR': 'Night Rapid Delivery',
    'NS': 'Tell Sell',
    'NO': 'Night OFS',
    'EZ': 'E Pallet',
    'SE': 'Special Events',
    'DD': 'Night Sideload'
}

# creating distribution_mode_desc column
visit['DISTRIBUTION_MODE_DESC'] = visit['DISTRIBUTION_MODE'].map(code_to_description)

In [11]:
# Mapping for shipping time
mapping_hours = {
    "24hrs": "24 Hours",
    "72hrs": "72 Hours",
    "48hrs": "48 Hours"
}

cutoff["SHIPPING_CONDITION_TIME"] = cutoff["SHIPPING_CONDITION_TIME"].map(mapping_hours)

In [12]:
cutoff[cutoff['PLANT_ID']=='G111']

,SALES_OFFICE,PLANT_ID,CUTOFFTIME__C,SHIPPING_CONDITION_TIME,DISTRIBUTION_MODE
0,"Draper, UT",G111,6:00:00 PM,72 Hours,Tell Sell
1,"Draper, UT",G111,6:00:00 PM,48 Hours,Tell Sell
2,"Draper, UT",G111,1:30:00 PM,24 Hours,Tell Sell
3,"Draper, UT",G111,3:30:00 PM,24 Hours,OFS
4,"Draper, UT",G111,3:30:00 PM,24 Hours,Bulk Distribution
5,"Draper, UT",G111,3:30:00 PM,24 Hours,Full Service
6,"Draper, UT",G111,3:30:00 PM,24 Hours,Sideload


In [13]:
master = pd.merge(visit, cutoff, left_on=["SALES_OFFICE","SHIPPING_CONDITIONS_DESC","DISTRIBUTION_MODE_DESC"], 
                  right_on=["PLANT_ID","SHIPPING_CONDITION_TIME","DISTRIBUTION_MODE"],how="left")



In [14]:
master["CUTOFFTIME__C"] = master["CUTOFFTIME__C"].fillna("5:00:00 PM")
master = master.drop(columns=["FREQUENCY", "ELT_TS","SNAPSHOT_DATE","SALES_OFFICE_y","SALES_OFFICE_DESC",
                              "DISTRIBUTION_MODE_x","SHIPPING_CONDITION_TIME","DISTRIBUTION_MODE_y","PLANT_ID"  ])



In [15]:
# Master table included data out of our needed range
master['ANCHOR_DATE'] = pd.to_datetime(master['ANCHOR_DATE'])
master_correct_date = master[(master['ANCHOR_DATE'] > pd.to_datetime('2024-06-05')) & 
       (master['ANCHOR_DATE'] < pd.to_datetime('2025-05-26'))].copy()

len(master_correct_date)

1780571

In [16]:
# UNIQUE ANCHOR_DATE FOR 1 SINGLE CUSTOMER
master = master.drop_duplicates(subset=['CUSTOMER_ID', 'ANCHOR_DATE']).sort_values('ANCHOR_DATE')
master[master['CUSTOMER_ID']==501738077].head()

# good example 501738077

,CUSTOMER_ID,ANCHOR_DATE,SALES_OFFICE_x,SHIPPING_CONDITIONS_DESC,WEEK_DAY_OF_ANCHOR_DATE,WINDOW_FREQUENCY,DISTRIBUTION_MODE_DESC,CUTOFFTIME__C
12454463,501738077,2024-10-23,G261,48 Hours,Wednesday,7,Sideload,4:30:00 PM
1358389,501738077,2024-12-17,G261,48 Hours,Tuesday,7,Rapid Delivery,4:30:00 PM
1531099,501738077,2025-02-07,G261,48 Hours,Friday,7,Rapid Delivery,4:30:00 PM


In [17]:
visit[visit['CUSTOMER_ID']==501738077].head(10)

,CUSTOMER_ID,FREQUENCY,ELT_TS,SNAPSHOT_DATE,ANCHOR_DATE,SALES_OFFICE,SALES_OFFICE_DESC,DISTRIBUTION_MODE,SHIPPING_CONDITIONS_DESC,WEEK_DAY_OF_ANCHOR_DATE,WINDOW_FREQUENCY,DISTRIBUTION_MODE_DESC
1359254,501738077,1.0,2024-11-20T02:37:13.345Z,2024-11-19,2024-12-17,G261,"Tempe, AZ",RD,48 Hours,Tuesday,7,Rapid Delivery
1361924,501738077,1.0,2024-11-21T02:42:04.855Z,2024-11-20,2024-12-17,G261,"Tempe, AZ",RD,48 Hours,Tuesday,7,Rapid Delivery
1368474,501738077,1.0,2024-11-25T02:39:03.172Z,2024-11-24,2024-12-17,G261,"Tempe, AZ",RD,48 Hours,Tuesday,7,Rapid Delivery
1372857,501738077,1.0,2024-11-29T02:36:12.919Z,2024-11-28,2024-12-17,G261,"Tempe, AZ",RD,48 Hours,Tuesday,7,Rapid Delivery
1375230,501738077,1.0,2024-11-30T02:38:31.799Z,2024-11-29,2024-12-17,G261,"Tempe, AZ",RD,48 Hours,Tuesday,7,Rapid Delivery
1388971,501738077,1.0,2024-12-08T02:37:23.073Z,2024-12-07,2024-12-17,G261,"Tempe, AZ",RD,48 Hours,Tuesday,7,Rapid Delivery
1419912,501738077,1.0,2024-12-27T02:34:27.209Z,2024-12-26,2024-12-17,G261,"Tempe, AZ",RD,48 Hours,Tuesday,7,Rapid Delivery
1446112,501738077,1.0,2025-01-13T02:32:38.509Z,2025-01-12,2024-12-17,G261,"Tempe, AZ",RD,48 Hours,Tuesday,7,Rapid Delivery
1446741,501738077,1.0,2025-01-13T15:00:03.885Z,2025-01-13,2024-12-17,G261,"Tempe, AZ",RD,48 Hours,Tuesday,7,Rapid Delivery
1461578,501738077,1.0,2025-01-22T02:34:52.257Z,2025-01-21,2024-12-17,G261,"Tempe, AZ",RD,48 Hours,Tuesday,7,Rapid Delivery


In [18]:
# Fill in between the unique ANCHOR_DATE with WINDOW_FREQUENCY

master_test = master.copy()

master_test["ANCHOR_DATE"] = pd.to_datetime(master_test["ANCHOR_DATE"])
master_test["WINDOW_FREQUENCY"] = pd.to_numeric(master_test["WINDOW_FREQUENCY"], errors="coerce").astype("Int64")

wd_map = {'monday':0,'tuesday':1,'wednesday':2,'thursday':3,'friday':4,'saturday':5,'sunday':6}
master_test["__wd"] = master_test["WEEK_DAY_OF_ANCHOR_DATE"].str.lower().map(wd_map)

# end of each policy segment = next original ANCHOR_DATE (per customer), else +365d
master_test = master_test.sort_values(["CUSTOMER_ID","ANCHOR_DATE"]).reset_index(drop=True)
master_test["__seg_end"] = master_test.groupby("CUSTOMER_ID")["ANCHOR_DATE"].shift(-1)
master_test["__seg_end"] = master_test["__seg_end"].fillna(master_test["ANCHOR_DATE"] + pd.Timedelta(days=365))

# first aligned NEXT for each original row
cand = master_test["ANCHOR_DATE"] + pd.to_timedelta(master_test["WINDOW_FREQUENCY"].fillna(0), unit="D")
off  = (master_test["__wd"] - cand.dt.weekday) % 7
first_next = cand + pd.to_timedelta(off, unit="D")

# build ranges (use int(step) only when notna)
ranges = [
    pd.date_range(start=fn, end=se - pd.Timedelta(days=1), freq=f"{int(step)}D")
    if (pd.notna(fn) and pd.notna(step) and fn < se) else pd.DatetimeIndex([])
    for fn, se, step in zip(first_next, master_test["__seg_end"], master_test["WINDOW_FREQUENCY"])
]

# explode
out = master_test.loc[master_test.index.repeat([len(r) for r in ranges])].copy()
out["ANCHOR_DATE"] = pd.DatetimeIndex([d for r in ranges for d in r])

# compute the next aligned date for each expanded row
next_cand = out["ANCHOR_DATE"] + pd.to_timedelta(out["WINDOW_FREQUENCY"], unit="D")
next_off  = (out["__wd"] - next_cand.dt.weekday) % 7
out["NEXT_ANCHOR_SAME_WD"] = next_cand + pd.to_timedelta(next_off, unit="D")

# include original rows with their NEXT
orig = master_test.copy()
orig["NEXT_ANCHOR_SAME_WD"] = first_next

final = (
    pd.concat([orig, out], ignore_index=True)
      .drop(columns=["__wd","__seg_end"])
      .sort_values(["CUSTOMER_ID","ANCHOR_DATE"])
      .reset_index(drop=True)
)


In [20]:
# Filter for what we the dates we need (from GA table min and max dates)
final = final[(final['ANCHOR_DATE'] > pd.to_datetime('2024-06-05')) & 
       (final['ANCHOR_DATE'] < pd.to_datetime('2025-05-26'))].copy()

In [21]:
len(final)

226145

In [22]:
final.isna().sum()

CUSTOMER_ID                  0
ANCHOR_DATE                  0
SALES_OFFICE_x               0
SHIPPING_CONDITIONS_DESC     0
WEEK_DAY_OF_ANCHOR_DATE      0
WINDOW_FREQUENCY             0
DISTRIBUTION_MODE_DESC      99
CUTOFFTIME__C                0
NEXT_ANCHOR_SAME_WD          0
dtype: int64

In [29]:
master = final.copy()

In [36]:
# Make a WINDOW_START column
master['ANCHOR_DATE'] = pd.to_datetime(master['ANCHOR_DATE'])
# Convert AM/PM time string to timedelta and add to datetime column
master['WINDOW_START'] = master['ANCHOR_DATE'] + \
    pd.to_datetime(master['CUTOFFTIME__C'], format='%I:%M:%S %p').apply(
        lambda t: pd.Timedelta(hours=t.hour, minutes=t.minute, seconds=t.second)
    )

In [ ]:
# Make the WINDOW_END column
master['NEXT_ANCHOR_SAME_WD'] = pd.to_datetime(master['NEXT_ANCHOR_SAME_WD'])

# Convert AM/PM time string to timedelta and add to datetime column
master['WINDOW_END'] = master['NEXT_ANCHOR_SAME_WD'] + \
    pd.to_datetime(master['CUTOFFTIME__C'], format='%I:%M:%S %p').apply(
        lambda t: pd.Timedelta(hours=t.hour, minutes=t.minute, seconds=t.second)
    )


In [59]:
# Same customer but now each rows are windows of order
with pd.option_context("display.max_rows", None, "display.max_columns", None):
    display(master[master['CUSTOMER_ID']==501738077])

,CUSTOMER_ID,ANCHOR_DATE,SALES_OFFICE_x,SHIPPING_CONDITIONS_DESC,WEEK_DAY_OF_ANCHOR_DATE,WINDOW_FREQUENCY,DISTRIBUTION_MODE_DESC,CUTOFFTIME__C,NEXT_ANCHOR_SAME_WD,WINDOW_END,WINDOW_START
980797,501738077,2024-10-23,G261,48 Hours,Wednesday,7,Sideload,4:30:00 PM,2024-10-30,2024-10-30 16:30:00,2024-10-23 16:30:00
980798,501738077,2024-10-30,G261,48 Hours,Wednesday,7,Sideload,4:30:00 PM,2024-11-06,2024-11-06 16:30:00,2024-10-30 16:30:00
980799,501738077,2024-11-06,G261,48 Hours,Wednesday,7,Sideload,4:30:00 PM,2024-11-13,2024-11-13 16:30:00,2024-11-06 16:30:00
980800,501738077,2024-11-13,G261,48 Hours,Wednesday,7,Sideload,4:30:00 PM,2024-11-20,2024-11-20 16:30:00,2024-11-13 16:30:00
980801,501738077,2024-11-20,G261,48 Hours,Wednesday,7,Sideload,4:30:00 PM,2024-11-27,2024-11-27 16:30:00,2024-11-20 16:30:00
980802,501738077,2024-11-27,G261,48 Hours,Wednesday,7,Sideload,4:30:00 PM,2024-12-04,2024-12-04 16:30:00,2024-11-27 16:30:00
980803,501738077,2024-12-04,G261,48 Hours,Wednesday,7,Sideload,4:30:00 PM,2024-12-11,2024-12-11 16:30:00,2024-12-04 16:30:00
980804,501738077,2024-12-11,G261,48 Hours,Wednesday,7,Sideload,4:30:00 PM,2024-12-18,2024-12-18 16:30:00,2024-12-11 16:30:00
980805,501738077,2024-12-17,G261,48 Hours,Tuesday,7,Rapid Delivery,4:30:00 PM,2024-12-24,2024-12-24 16:30:00,2024-12-17 16:30:00
980806,501738077,2024-12-24,G261,48 Hours,Tuesday,7,Rapid Delivery,4:30:00 PM,2024-12-31,2024-12-31 16:30:00,2024-12-24 16:30:00


In [22]:
#master.to_csv("master.csv")

# Adding Ga into master?



In [39]:
# Format ITEMS correctly
import ast

ga['ITEMS_parsed'] = ga['ITEMS'].dropna().apply(
    lambda x: ast.literal_eval(x) if x != '[]' else []
)


In [ ]:
# Care only about carts events
ga_cart_events = ga[ga['EVENT_NAME'].isin(['purchase', 'add_to_cart', 'remove_from_cart'])].sort_values('EVENT_TIMESTAMP')

,CUSTOMER_ID,EVENT_DATE,EVENT_TIMESTAMP,EVENT_NAME,DEVICE_CATEGORY,DEVICE_MOBILE_BRAND_NAME,DEVICE_OPERATING_SYSTEM,EVENT_PAGE_NAME,EVENT_PAGE_TITLE,ITEMS,ITEMS_parsed
25196,501738077,2025-01-10,2025-01-10T21:37:41.780Z,add_to_cart,desktop,Google,Chrome OS,MyCoke Dashboard,Category: All Products,"[{""item_id"":""119826"",""quantity"":""3""}]","[{'item_id': '119826', 'quantity': '3'}]"
3332185,501738077,2025-01-10,2025-01-10T21:37:50.056Z,add_to_cart,desktop,Google,Chrome OS,MyCoke Dashboard,Category: All Products,"[{""item_id"":""112259"",""quantity"":""2""}]","[{'item_id': '112259', 'quantity': '2'}]"
943427,501738077,2025-01-10,2025-01-10T21:38:24.647Z,add_to_cart,desktop,Google,Chrome OS,MyCoke Dashboard,Category: All Products,"[{""item_id"":""117803"",""quantity"":""2""}]","[{'item_id': '117803', 'quantity': '2'}]"
1675596,501738077,2025-01-10,2025-01-10T21:39:19.325Z,add_to_cart,desktop,Google,Chrome OS,MyCoke Dashboard,Category: All Products,"[{""item_id"":""101728"",""quantity"":""2""}]","[{'item_id': '101728', 'quantity': '2'}]"
1646795,501738077,2025-01-10,2025-01-10T21:39:48.269Z,add_to_cart,desktop,Google,Chrome OS,MyCoke Dashboard,Category: All Products,"[{""item_id"":""102752"",""quantity"":""2""}]","[{'item_id': '102752', 'quantity': '2'}]"


In [44]:
# Much smaller data
print(len(ga))
print(len(ga_cart_events))

# What is a good range of time for our date?
print(ga_cart_events['EVENT_TIMESTAMP'].min())
print(ga_cart_events['EVENT_TIMESTAMP'].max())

3704088
275865
2024-06-04T22:15:12.695Z
2025-05-26T03:59:27.124Z


In [ ]:
# Join ga-purcahse_events with customer to get local time zone
ga_events_with_timezone = ga_cart_events.merge(customer, left_on="CUSTOMER_ID", right_on="CUSTOMER_NUMBER", how='left')

,CUSTOMER_ID,EVENT_DATE,EVENT_TIMESTAMP,EVENT_NAME,DEVICE_CATEGORY,DEVICE_MOBILE_BRAND_NAME,DEVICE_OPERATING_SYSTEM,EVENT_PAGE_NAME,EVENT_PAGE_TITLE,ITEMS,ITEMS_parsed,SALES_OFFICE,CUSTOMER_NUMBER,SALES_OFFICE_DESCRIPTION,DISTRIBUTION_MODE_DESCRIPTION,SHIPPING_CONDITIONS_DESCRIPTION,COLD_DRINK_CHANNEL_DESCRIPTION,CUSTOMER_SUB_TRADE_CHANNEL_DESCRIPTION
0,501282634,2024-06-04,2024-06-04T22:15:12.695Z,add_to_cart,desktop,Apple,Macintosh,MyCoke Orders,Order Builder,"[{""item_id"":""104148"",""quantity"":""2""}]","[{'item_id': '104148', 'quantity': '2'}]",G267,501282634,"Albuquerque, NM",Rapid Delivery,48 Hours,Restaurant,Pizza Fast Food
1,501282634,2024-06-04,2024-06-04T22:16:04.891Z,remove_from_cart,desktop,Apple,Macintosh,MyCoke Orders - Cart,Cart,"[{""item_id"":""104148"",""quantity"":""0""}]","[{'item_id': '104148', 'quantity': '0'}]",G267,501282634,"Albuquerque, NM",Rapid Delivery,48 Hours,Restaurant,Pizza Fast Food
2,501282634,2024-06-04,2024-06-04T22:18:22.810Z,add_to_cart,desktop,Apple,Macintosh,MyCoke Orders,Order Builder,"[{""item_id"":""103996"",""quantity"":""2""}]","[{'item_id': '103996', 'quantity': '2'}]",G267,501282634,"Albuquerque, NM",Rapid Delivery,48 Hours,Restaurant,Pizza Fast Food
3,501282634,2024-06-04,2024-06-04T22:26:59.003Z,remove_from_cart,desktop,Apple,Macintosh,MyCoke Orders - Cart,Cart,"[{""item_id"":""103996"",""quantity"":""1""}]","[{'item_id': '103996', 'quantity': '1'}]",G267,501282634,"Albuquerque, NM",Rapid Delivery,48 Hours,Restaurant,Pizza Fast Food
4,501282634,2024-06-04,2024-06-04T22:26:59.003Z,remove_from_cart,desktop,Apple,Macintosh,MyCoke Orders - Cart,Cart,"[{""item_id"":""103996"",""quantity"":""1""}]","[{'item_id': '103996', 'quantity': '1'}]",G267,501282634,"Albuquerque, NM",Rapid Delivery,48 Hours,Restaurant,Pizza Fast Food


In [ ]:
# Only need to map to handful of locations
ga_events_with_timezone['SALES_OFFICE_DESCRIPTION'].unique()

array(['Albuquerque, NM', 'Richfield, UT', 'Colorado Springs, CO',
       'Denver, CO', 'Draper, UT', 'Boise, ID', 'Johnstown, CO',
       'Bellevue, WA', 'Wenatchee, WA', 'Ogden, UT', 'Glendale, AZ',
       'Bend, OR', 'Twin Falls, ID', 'Tacoma, WA', 'Spokane, WA',
       'Lewiston, ID', 'Alamosa, CO', 'Grand Junction, CO',
       'Pocatello, ID', 'Reno, NV', 'Pendleton, OR', 'Pueblo, CO',
       'Idaho Falls, ID', 'Tempe, AZ', 'Flagstaff, AZ', 'Arlington, WA',
       'Logan, UT', 'Bremerton, WA', 'Kingman, AZ', 'Wilsonville, OR',
       'Eugene, OR', 'Show Low, AZ', 'Prescott, AZ',
       'Glenwood Springs, CO', 'Price, UT', 'Tucson, AZ', 'Cheyenne, WY',
       'Elko, NV', 'Walla Walla, WA', 'Yuma, AZ', 'Huachuca City, AZ',
       'Scottsbluff, NE', 'LaGrande, OR'], dtype=object)

In [47]:

# --- 1. Extract STATE from SALES_OFFICE_DESCRIPTION ---
ga_events_with_timezone["STATE"] = ga_events_with_timezone["SALES_OFFICE_DESCRIPTION"].str.extract(r",\s*([A-Z]{2})")

# --- 2. Map state → timezone ---
state_tz_map = {
    "WA": "America/Los_Angeles",
    "OR": "America/Los_Angeles",
    "NV": "America/Los_Angeles",
    "CA": "America/Los_Angeles",
    "AZ": "America/Phoenix",   # no DST
    "UT": "America/Denver",
    "CO": "America/Denver",
    "NM": "America/Denver",
    "ID": "America/Boise",
    "MT": "America/Denver",
    "WY": "America/Denver",
    "NE": "America/Chicago",
}
ga_events_with_timezone["TIMEZONE"] = ga_events_with_timezone["STATE"].map(state_tz_map)

# --- 3. Parse EVENT_TIMESTAMP as tz-aware UTC ---
ga_events_with_timezone["EVENT_TIMESTAMP_UTC"] = pd.to_datetime(
    ga_events_with_timezone["EVENT_TIMESTAMP"], utc=True, errors="coerce"
)

# --- 4. Convert to local time per timezone group and drop tz immediately ---
ga_events_with_timezone["EVENT_TIMESTAMP_LOCAL"] = pd.NaT

for tz, idx in ga_events_with_timezone.groupby("TIMEZONE").groups.items():
    if pd.isna(tz):
        continue
    ga_events_with_timezone.loc[idx, "EVENT_TIMESTAMP_LOCAL"] = (
        ga_events_with_timezone.loc[idx, "EVENT_TIMESTAMP_UTC"]
          .dt.tz_convert(tz)        # local tz-aware
          .dt.tz_localize(None)     # drop tz → tz-naive local clock
    )

# --- 5. Clean up helper column if you don’t need it ---
ga_events_with_timezone.drop(columns=["EVENT_TIMESTAMP_UTC"], inplace=True)




In [48]:
ga_events_with_timezone.head()

,CUSTOMER_ID,EVENT_DATE,EVENT_TIMESTAMP,EVENT_NAME,DEVICE_CATEGORY,DEVICE_MOBILE_BRAND_NAME,DEVICE_OPERATING_SYSTEM,EVENT_PAGE_NAME,EVENT_PAGE_TITLE,ITEMS,...,SALES_OFFICE,CUSTOMER_NUMBER,SALES_OFFICE_DESCRIPTION,DISTRIBUTION_MODE_DESCRIPTION,SHIPPING_CONDITIONS_DESCRIPTION,COLD_DRINK_CHANNEL_DESCRIPTION,CUSTOMER_SUB_TRADE_CHANNEL_DESCRIPTION,STATE,TIMEZONE,EVENT_TIMESTAMP_LOCAL
0,501282634,2024-06-04,2024-06-04T22:15:12.695Z,add_to_cart,desktop,Apple,Macintosh,MyCoke Orders,Order Builder,"[{""item_id"":""104148"",""quantity"":""2""}]",...,G267,501282634,"Albuquerque, NM",Rapid Delivery,48 Hours,Restaurant,Pizza Fast Food,NM,America/Denver,2024-06-04 16:15:12.695
1,501282634,2024-06-04,2024-06-04T22:16:04.891Z,remove_from_cart,desktop,Apple,Macintosh,MyCoke Orders - Cart,Cart,"[{""item_id"":""104148"",""quantity"":""0""}]",...,G267,501282634,"Albuquerque, NM",Rapid Delivery,48 Hours,Restaurant,Pizza Fast Food,NM,America/Denver,2024-06-04 16:16:04.891
2,501282634,2024-06-04,2024-06-04T22:18:22.810Z,add_to_cart,desktop,Apple,Macintosh,MyCoke Orders,Order Builder,"[{""item_id"":""103996"",""quantity"":""2""}]",...,G267,501282634,"Albuquerque, NM",Rapid Delivery,48 Hours,Restaurant,Pizza Fast Food,NM,America/Denver,2024-06-04 16:18:22.810
3,501282634,2024-06-04,2024-06-04T22:26:59.003Z,remove_from_cart,desktop,Apple,Macintosh,MyCoke Orders - Cart,Cart,"[{""item_id"":""103996"",""quantity"":""1""}]",...,G267,501282634,"Albuquerque, NM",Rapid Delivery,48 Hours,Restaurant,Pizza Fast Food,NM,America/Denver,2024-06-04 16:26:59.003
4,501282634,2024-06-04,2024-06-04T22:26:59.003Z,remove_from_cart,desktop,Apple,Macintosh,MyCoke Orders - Cart,Cart,"[{""item_id"":""103996"",""quantity"":""1""}]",...,G267,501282634,"Albuquerque, NM",Rapid Delivery,48 Hours,Restaurant,Pizza Fast Food,NM,America/Denver,2024-06-04 16:26:59.003


In [49]:
master.head()

,CUSTOMER_ID,ANCHOR_DATE,SALES_OFFICE_x,SHIPPING_CONDITIONS_DESC,WEEK_DAY_OF_ANCHOR_DATE,WINDOW_FREQUENCY,DISTRIBUTION_MODE_DESC,CUTOFFTIME__C,NEXT_ANCHOR_SAME_WD,WINDOW_END,WINDOW_START
219,500245685,2024-06-10,G111,48 Hours,Monday,7,OFS,5:00:00 PM,2024-06-17,2024-06-17 17:00:00,2024-06-10 17:00:00
220,500245685,2024-06-17,G111,48 Hours,Monday,7,OFS,5:00:00 PM,2024-06-24,2024-06-24 17:00:00,2024-06-17 17:00:00
221,500245685,2024-06-24,G111,48 Hours,Monday,7,OFS,5:00:00 PM,2024-07-01,2024-07-01 17:00:00,2024-06-24 17:00:00
222,500245685,2024-07-01,G111,48 Hours,Monday,7,OFS,5:00:00 PM,2024-07-08,2024-07-08 17:00:00,2024-07-01 17:00:00
223,500245685,2024-07-08,G111,48 Hours,Monday,7,OFS,5:00:00 PM,2024-07-15,2024-07-15 17:00:00,2024-07-08 17:00:00


In [63]:
# Merge master and ga_events_with_timezone

merged = master.merge(
    ga_events_with_timezone,
    on='CUSTOMER_ID',
    how='left'
)

# Delete duplicated events that felt outside windows
merged = merged[
    (merged['EVENT_TIMESTAMP_LOCAL'] >= merged['WINDOW_START']) &
    (merged['EVENT_TIMESTAMP_LOCAL'] < merged['WINDOW_END'])
].sort_values('WINDOW_START')

# Delete duplicated columns
merged = merged.drop(columns=['SALES_OFFICE_x','NEXT_ANCHOR_SAME_WD','EVENT_TIMESTAMP','ITEMS','CUSTOMER_NUMBER'])

In [79]:
with pd.option_context("display.max_rows", None, "display.max_columns", None):
    display(merged[merged['CUSTOMER_ID']==500245797])

,CUSTOMER_ID,ANCHOR_DATE,SHIPPING_CONDITIONS_DESC,WEEK_DAY_OF_ANCHOR_DATE,WINDOW_FREQUENCY,DISTRIBUTION_MODE_DESC,CUTOFFTIME__C,WINDOW_END,WINDOW_START,EVENT_DATE,EVENT_NAME,DEVICE_CATEGORY,DEVICE_MOBILE_BRAND_NAME,DEVICE_OPERATING_SYSTEM,EVENT_PAGE_NAME,EVENT_PAGE_TITLE,ITEMS_parsed,SALES_OFFICE,SALES_OFFICE_DESCRIPTION,DISTRIBUTION_MODE_DESCRIPTION,SHIPPING_CONDITIONS_DESCRIPTION,COLD_DRINK_CHANNEL_DESCRIPTION,CUSTOMER_SUB_TRADE_CHANNEL_DESCRIPTION,STATE,TIMEZONE,EVENT_TIMESTAMP_LOCAL
9857,500245797,2024-07-29,48 Hours,Monday,7,OFS,5:00:00 PM,2024-08-05 17:00:00,2024-07-29 17:00:00,2024-07-30,add_to_cart,desktop,Apple,Macintosh,NaN,Search,"[{'item_id': '150936', 'quantity': '2'}]",G111,"Draper, UT",OFS,48 Hours,Workplace,Business Service,UT,America/Denver,2024-07-30 14:57:12.081
9858,500245797,2024-07-29,48 Hours,Monday,7,OFS,5:00:00 PM,2024-08-05 17:00:00,2024-07-29 17:00:00,2024-07-30,add_to_cart,desktop,Apple,Macintosh,NaN,Category: All Products,"[{'item_id': '157760', 'quantity': '1'}]",G111,"Draper, UT",OFS,48 Hours,Workplace,Business Service,UT,America/Denver,2024-07-30 14:57:35.635
9859,500245797,2024-07-29,48 Hours,Monday,7,OFS,5:00:00 PM,2024-08-05 17:00:00,2024-07-29 17:00:00,2024-07-30,add_to_cart,desktop,Apple,Macintosh,NaN,Category: All Products,"[{'item_id': '150937', 'quantity': '1'}]",G111,"Draper, UT",OFS,48 Hours,Workplace,Business Service,UT,America/Denver,2024-07-30 14:58:11.074
9860,500245797,2024-07-29,48 Hours,Monday,7,OFS,5:00:00 PM,2024-08-05 17:00:00,2024-07-29 17:00:00,2024-07-30,add_to_cart,desktop,Apple,Macintosh,NaN,Category: All Products,"[{'item_id': '150926', 'quantity': '1'}]",G111,"Draper, UT",OFS,48 Hours,Workplace,Business Service,UT,America/Denver,2024-07-30 14:58:46.135
9861,500245797,2024-07-29,48 Hours,Monday,7,OFS,5:00:00 PM,2024-08-05 17:00:00,2024-07-29 17:00:00,2024-07-30,purchase,desktop,Apple,Macintosh,MyCoke Orders - Purchase Success,Category: All Products,"[{'item_id': '150926', 'quantity': '1'}, {'ite...",G111,"Draper, UT",OFS,48 Hours,Workplace,Business Service,UT,America/Denver,2024-07-30 15:02:45.810
10044,500245797,2024-08-05,48 Hours,Monday,7,OFS,5:00:00 PM,2024-08-12 17:00:00,2024-08-05 17:00:00,2024-08-07,add_to_cart,desktop,Apple,Macintosh,NaN,Search,"[{'item_id': '150936', 'quantity': '3'}]",G111,"Draper, UT",OFS,48 Hours,Workplace,Business Service,UT,America/Denver,2024-08-07 15:25:54.263
10045,500245797,2024-08-05,48 Hours,Monday,7,OFS,5:00:00 PM,2024-08-12 17:00:00,2024-08-05 17:00:00,2024-08-07,add_to_cart,desktop,Apple,Macintosh,NaN,Search,"[{'item_id': '157761', 'quantity': '1'}]",G111,"Draper, UT",OFS,48 Hours,Workplace,Business Service,UT,America/Denver,2024-08-07 15:26:14.286
10046,500245797,2024-08-05,48 Hours,Monday,7,OFS,5:00:00 PM,2024-08-12 17:00:00,2024-08-05 17:00:00,2024-08-07,add_to_cart,desktop,Apple,Macintosh,NaN,Search,"[{'item_id': '151219', 'quantity': '1'}]",G111,"Draper, UT",OFS,48 Hours,Workplace,Business Service,UT,America/Denver,2024-08-07 15:26:25.458
10047,500245797,2024-08-05,48 Hours,Monday,7,OFS,5:00:00 PM,2024-08-12 17:00:00,2024-08-05 17:00:00,2024-08-07,add_to_cart,desktop,Apple,Macintosh,NaN,Category: All Products,"[{'item_id': '150915', 'quantity': '1'}]",G111,"Draper, UT",OFS,48 Hours,Workplace,Business Service,UT,America/Denver,2024-08-07 15:27:24.541
10048,500245797,2024-08-05,48 Hours,Monday,7,OFS,5:00:00 PM,2024-08-12 17:00:00,2024-08-05 17:00:00,2024-08-07,add_to_cart,desktop,Apple,Macintosh,NaN,Category: All Products,"[{'item_id': '150925', 'quantity': '1'}]",G111,"Draper, UT",OFS,48 Hours,Workplace,Business Service,UT,America/Denver,2024-08-07 15:28:26.748


In [65]:
merged_copy = merged.copy()

# --- 1) Row-level flags ---
def has_items(x):
    if isinstance(x, (list, tuple)):
        return len(x) > 0
    if isinstance(x, str):
        return len(x.strip()) > 2  # crude JSON-ish non-empty
    return False

merged_copy['is_purchase']    = merged_copy['EVENT_NAME'].eq('purchase')
merged_copy['has_cart_items'] = merged_copy['ITEMS_parsed'].apply(has_items)


# merged['has_cart_items'] = merged['has_cart_items'] & cart_like

win_cols = ['CUSTOMER_ID', 'WINDOW_START', 'WINDOW_END']

# Sort so "last" means max timestamp within window
merged_copy = merged_copy.sort_values(win_cols + ['EVENT_TIMESTAMP_LOCAL'])

# --- 2) Per-window summary ---
agg = merged_copy.groupby(win_cols, as_index=False).agg(
    has_any_purchase=('is_purchase', 'any'),
    has_any_cart=('has_cart_items', 'any'),
    last_ts=('EVENT_TIMESTAMP_LOCAL', 'max')
)

# --- 3) Keep one row per window according to rules ---
# 3a) If any purchase in window -> keep LAST purchase only, abandoned=0
last_purch = (
    merged_copy[merged_copy['is_purchase']]
    .merge(agg[agg['has_any_purchase']][win_cols + ['last_ts']],
           on=win_cols, how='inner')
)
keep_purchase = last_purch[
    last_purch['EVENT_TIMESTAMP_LOCAL'].eq(last_purch['last_ts'])
].copy()
keep_purchase['abandoned'] = 0

# 3b) If NO purchase but HAS cart items -> keep LAST event of window, abandoned=1
no_purch_windows = agg[(~agg['has_any_purchase']) & (agg['has_any_cart'])][win_cols + ['last_ts']]
last_event_no_purch = merged_copy.merge(no_purch_windows, on=win_cols, how='inner')
keep_abandoned = last_event_no_purch[
    last_event_no_purch['EVENT_TIMESTAMP_LOCAL'].eq(last_event_no_purch['last_ts'])
].copy()
keep_abandoned['abandoned'] = 1

# --- 4) Overwrite `merged_copy` with the kept rows; drop everything else ---
kept = pd.concat([keep_purchase, keep_abandoned], ignore_index=True)

# If you want to drop the helper columns from the final `merged_copy`, do it here:
cols_drop = ['is_purchase', 'has_cart_items', 'last_ts', 'has_any_purchase', 'has_any_cart']
cols_drop = [c for c in cols_drop if c in kept.columns]
merged_copy = kept.drop(columns=cols_drop).sort_values(win_cols + ['EVENT_TIMESTAMP_LOCAL']).reset_index(drop=True)

# `merged_copy` now has exactly one row per window with an `abandoned` 0/1 flag.


In [ ]:
with pd.option_context("display.max_rows", None, "display.max_columns", None):
    display(merged_copy[merged_copy['CUSTOMER_ID']==500245797])

,CUSTOMER_ID,ANCHOR_DATE,SHIPPING_CONDITIONS_DESC,WEEK_DAY_OF_ANCHOR_DATE,WINDOW_FREQUENCY,DISTRIBUTION_MODE_DESC,CUTOFFTIME__C,WINDOW_END,WINDOW_START,EVENT_DATE,EVENT_NAME,DEVICE_CATEGORY,DEVICE_MOBILE_BRAND_NAME,DEVICE_OPERATING_SYSTEM,EVENT_PAGE_NAME,EVENT_PAGE_TITLE,ITEMS_parsed,SALES_OFFICE,SALES_OFFICE_DESCRIPTION,DISTRIBUTION_MODE_DESCRIPTION,SHIPPING_CONDITIONS_DESCRIPTION,COLD_DRINK_CHANNEL_DESCRIPTION,CUSTOMER_SUB_TRADE_CHANNEL_DESCRIPTION,STATE,TIMEZONE,EVENT_TIMESTAMP_LOCAL,abandoned
22313,501738077,2025-01-07,48 Hours,Tuesday,7,Rapid Delivery,4:30:00 PM,2025-01-14 16:30:00,2025-01-07 16:30:00,2025-01-10,purchase,desktop,Google,Chrome OS,MyCoke Orders - Purchase Success,Order,"[{'item_id': '132540', 'quantity': '2'}, {'ite...",G261,"Tempe, AZ",Rapid Delivery,48 Hours,Restaurant,Pizza Fast Food,AZ,America/Phoenix,2025-01-10 15:34:02.685,0
22314,501738077,2025-01-21,48 Hours,Tuesday,7,Rapid Delivery,4:30:00 PM,2025-01-28 16:30:00,2025-01-21 16:30:00,2025-01-24,purchase,desktop,Google,Chrome OS,MyCoke Orders - Purchase Success,Order,"[{'item_id': '132606', 'quantity': '1'}, {'ite...",G261,"Tempe, AZ",Rapid Delivery,48 Hours,Restaurant,Pizza Fast Food,AZ,America/Phoenix,2025-01-24 11:44:40.523,0
22315,501738077,2025-01-28,48 Hours,Tuesday,7,Rapid Delivery,4:30:00 PM,2025-02-04 16:30:00,2025-01-28 16:30:00,2025-01-31,purchase,desktop,Google,Chrome OS,MyCoke Orders - Purchase Success,Order,"[{'item_id': '119827', 'quantity': '1'}, {'ite...",G261,"Tempe, AZ",Rapid Delivery,48 Hours,Restaurant,Pizza Fast Food,AZ,America/Phoenix,2025-01-31 14:39:15.106,0
22316,501738077,2025-02-07,48 Hours,Friday,7,Rapid Delivery,4:30:00 PM,2025-02-14 16:30:00,2025-02-07 16:30:00,2025-02-14,purchase,desktop,Google,Chrome OS,MyCoke Orders - Purchase Success,Order,"[{'item_id': '119826', 'quantity': '4'}, {'ite...",G261,"Tempe, AZ",Rapid Delivery,48 Hours,Restaurant,Pizza Fast Food,AZ,America/Phoenix,2025-02-14 15:02:08.961,0
22317,501738077,2025-02-28,48 Hours,Friday,7,Rapid Delivery,4:30:00 PM,2025-03-07 16:30:00,2025-02-28 16:30:00,2025-03-04,purchase,desktop,Google,Chrome OS,MyCoke Orders - Purchase Success,Order,"[{'item_id': '117803', 'quantity': '2'}, {'ite...",G261,"Tempe, AZ",Rapid Delivery,48 Hours,Restaurant,Pizza Fast Food,AZ,America/Phoenix,2025-03-04 15:00:07.797,0
22318,501738077,2025-03-21,48 Hours,Friday,7,Rapid Delivery,4:30:00 PM,2025-03-28 16:30:00,2025-03-21 16:30:00,2025-03-26,purchase,desktop,Google,Chrome OS,MyCoke Orders - Purchase Success,Order,"[{'item_id': '132551', 'quantity': '1'}, {'ite...",G261,"Tempe, AZ",Rapid Delivery,48 Hours,Restaurant,Pizza Fast Food,AZ,America/Phoenix,2025-03-26 13:18:15.474,0
22319,501738077,2025-04-04,48 Hours,Friday,7,Rapid Delivery,4:30:00 PM,2025-04-11 16:30:00,2025-04-04 16:30:00,2025-04-09,purchase,desktop,Google,Chrome OS,MyCoke Orders - Purchase Success,Order,"[{'item_id': '102748', 'quantity': '2'}, {'ite...",G261,"Tempe, AZ",Rapid Delivery,48 Hours,Restaurant,Pizza Fast Food,AZ,America/Phoenix,2025-04-09 12:51:55.422,0
22320,501738077,2025-05-02,48 Hours,Friday,7,Rapid Delivery,4:30:00 PM,2025-05-09 16:30:00,2025-05-02 16:30:00,2025-05-07,purchase,desktop,Google,Chrome OS,MyCoke Orders - Purchase Success,Order,"[{'item_id': '132532', 'quantity': '3'}, {'ite...",G261,"Tempe, AZ",Rapid Delivery,48 Hours,Restaurant,Pizza Fast Food,AZ,America/Phoenix,2025-05-07 14:47:11.617,0


In [75]:
len(merged_copy)

31628

In [78]:
merged_copy[merged_copy['abandoned']==1]

,CUSTOMER_ID,ANCHOR_DATE,SHIPPING_CONDITIONS_DESC,WEEK_DAY_OF_ANCHOR_DATE,WINDOW_FREQUENCY,DISTRIBUTION_MODE_DESC,CUTOFFTIME__C,WINDOW_END,WINDOW_START,EVENT_DATE,...,SALES_OFFICE,SALES_OFFICE_DESCRIPTION,DISTRIBUTION_MODE_DESCRIPTION,SHIPPING_CONDITIONS_DESCRIPTION,COLD_DRINK_CHANNEL_DESCRIPTION,CUSTOMER_SUB_TRADE_CHANNEL_DESCRIPTION,STATE,TIMEZONE,EVENT_TIMESTAMP_LOCAL,abandoned
18,500245738,2024-11-08,24 Hours,Friday,7,OFS,3:30:00 PM,2024-11-15 15:30:00,2024-11-08 15:30:00,2024-11-13,...,G111,"Draper, UT",OFS,48 Hours,Distributor,Food Distributor,UT,America/Denver,2024-11-13 12:06:09.492,1
29,500245738,2025-01-29,48 Hours,Wednesday,7,OFS,5:00:00 PM,2025-02-05 17:00:00,2025-01-29 17:00:00,2025-02-03,...,G111,"Draper, UT",OFS,48 Hours,Distributor,Food Distributor,UT,America/Denver,2025-02-03 15:06:59.831,1
53,500245797,2024-09-30,48 Hours,Monday,7,OFS,5:00:00 PM,2024-10-07 17:00:00,2024-09-30 17:00:00,2024-10-01,...,G111,"Draper, UT",OFS,48 Hours,Workplace,Business Service,UT,America/Denver,2024-10-01 17:25:11.299,1
55,500245797,2024-10-28,48 Hours,Monday,7,OFS,5:00:00 PM,2024-11-04 17:00:00,2024-10-28 17:00:00,2024-10-31,...,G111,"Draper, UT",OFS,48 Hours,Workplace,Business Service,UT,America/Denver,2024-10-31 13:33:53.105,1
58,500245797,2024-12-27,48 Hours,Friday,28,OFS,5:00:00 PM,2025-01-24 17:00:00,2024-12-27 17:00:00,2025-01-10,...,G111,"Draper, UT",OFS,48 Hours,Workplace,Business Service,UT,America/Denver,2025-01-10 10:41:35.305,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31592,600866904,2024-10-03,48 Hours,Thursday,7,OFS,5:00:00 PM,2024-10-10 17:00:00,2024-10-03 17:00:00,2024-10-07,...,G291,"Bellevue, WA",OFS,48 Hours,Attraction,Recreation Center,WA,America/Los_Angeles,2024-10-07 14:20:23.246,1
31594,600866904,2024-10-17,48 Hours,Thursday,7,OFS,5:00:00 PM,2024-10-24 17:00:00,2024-10-17 17:00:00,2024-10-22,...,G291,"Bellevue, WA",OFS,48 Hours,Attraction,Recreation Center,WA,America/Los_Angeles,2024-10-22 17:02:55.808,1
31596,600866904,2024-11-07,48 Hours,Thursday,7,OFS,5:00:00 PM,2024-11-14 17:00:00,2024-11-07 17:00:00,2024-11-12,...,G291,"Bellevue, WA",OFS,48 Hours,Attraction,Recreation Center,WA,America/Los_Angeles,2024-11-12 11:01:58.038,1
31605,600975210,2024-11-13,48 Hours,Wednesday,7,OFS,3:30:00 PM,2024-11-20 15:30:00,2024-11-13 15:30:00,2024-11-17,...,G282,"Wenatchee, WA",OFS,48 Hours,Restaurant,Fast Food,WA,America/Los_Angeles,2024-11-17 19:29:07.875,1


In [76]:
5168/31628

0.1633995194131782

In [82]:
(merged_copy['abandoned']).mean()

0.1633995194131782

In [135]:
import pandas as pd

# copies (optional) 
e = ga_purchase_with_timezone.copy()
m = master.copy()

# 1) Ensure datetime dtypes
e["EVENT_TIMESTAMP_LOCAL"] = pd.to_datetime(e["EVENT_TIMESTAMP_LOCAL"], errors="coerce")
m["ANCHOR_DATE"]  = pd.to_datetime(m["ANCHOR_DATE"],  errors="coerce")
m["WINDOW_END"]   = pd.to_datetime(m["WINDOW_END"],   errors="coerce")

# 2) Keep only rows with the keys we need
e2 = e.dropna(subset=["CUSTOMER_ID", "EVENT_TIMESTAMP_LOCAL"])
m2 = m.dropna(subset=["CUSTOMER_ID", "ANCHOR_DATE", "WINDOW_END"])

# 3) Plain merge on CUSTOMER_ID (no asof), then filter by the window
cand = e2.merge(
    m2[["CUSTOMER_ID","ANCHOR_DATE","WINDOW_END"]], 
    on="CUSTOMER_ID", 
    how="inner"
)

in_window = (
    (cand["EVENT_TIMESTAMP_LOCAL"] >= cand["ANCHOR_DATE"]) &
    (cand["EVENT_TIMESTAMP_LOCAL"] <= cand["WINDOW_END"])
)
cand = cand[in_window]

# 4) If multiple windows match an event, keep the one with the latest ANCHOR_DATE
#    (group by event identity: CUSTOMER_ID + EVENT_TIMESTAMP_LOCAL)
idx = cand.groupby(["CUSTOMER_ID","EVENT_TIMESTAMP_LOCAL"])["ANCHOR_DATE"].idxmax()
asgn = cand.loc[idx]

# 5) LEFT join back to master so every window shows up (duplicates if multiple events per window)
result = m2.merge(
    asgn.drop(columns=["WINDOW_END"]),          # avoid duplicate col
    on=["CUSTOMER_ID","ANCHOR_DATE"],
    how="left"
)

# 'result' is your master windows with events that fell inside [ANCHOR_DATE, WINDOW_END]


In [141]:
window_and_purchase = result.drop(columns=['CUTOFFTIME__C', 'NEXT_ANCHOR_SAME_WD','EVENT_TIMESTAMP','EVENT_DATE','ITEMS','SALES_OFFICE_x'])

In [142]:
with pd.option_context("display.max_rows", None, "display.max_columns", None):
    display(window_and_purchase[window_and_purchase['CUSTOMER_ID']==501738077])

,CUSTOMER_ID,ANCHOR_DATE,SHIPPING_CONDITIONS_DESC,WEEK_DAY_OF_ANCHOR_DATE,WINDOW_FREQUENCY,DISTRIBUTION_MODE_DESC,WINDOW_END,EVENT_NAME,DEVICE_CATEGORY,DEVICE_MOBILE_BRAND_NAME,DEVICE_OPERATING_SYSTEM,EVENT_PAGE_NAME,EVENT_PAGE_TITLE,ITEMS_parsed,SALES_OFFICE,CUSTOMER_NUMBER,SALES_OFFICE_DESCRIPTION,DISTRIBUTION_MODE_DESCRIPTION,SHIPPING_CONDITIONS_DESCRIPTION,COLD_DRINK_CHANNEL_DESCRIPTION,CUSTOMER_SUB_TRADE_CHANNEL_DESCRIPTION,STATE,TIMEZONE,EVENT_TIMESTAMP_LOCAL
165387,501738077,2024-10-23,48 Hours,Wednesday,7,Sideload,2024-10-30 16:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
165388,501738077,2024-10-30,48 Hours,Wednesday,7,Sideload,2024-11-06 16:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
165389,501738077,2024-11-06,48 Hours,Wednesday,7,Sideload,2024-11-13 16:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
165390,501738077,2024-11-13,48 Hours,Wednesday,7,Sideload,2024-11-20 16:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
165391,501738077,2024-11-20,48 Hours,Wednesday,7,Sideload,2024-11-27 16:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
165392,501738077,2024-11-27,48 Hours,Wednesday,7,Sideload,2024-12-04 16:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
165393,501738077,2024-12-04,48 Hours,Wednesday,7,Sideload,2024-12-11 16:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
165394,501738077,2024-12-11,48 Hours,Wednesday,7,Sideload,2024-12-18 16:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
165395,501738077,2024-12-17,48 Hours,Tuesday,7,Rapid Delivery,2024-12-24 16:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
165396,501738077,2024-12-24,48 Hours,Tuesday,7,Rapid Delivery,2024-12-31 16:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT


In [146]:
# For our example the len of window above from master is the same as the merged table, left join worked

print(len(window_and_purchase[window_and_purchase['CUSTOMER_ID']==501738077]))
print(len(master[master['CUSTOMER_ID']==501738077]))

32
32


In [150]:
len(window_and_purchase)

230976

In [149]:
window_and_purchase['EVENT_NAME'].isna().sum()

198972

In [152]:
window_and_purchase['EVENT_NAME'].notna().sum()


32004

In [151]:
# 86% of rows are NA! something not right?
198972/230976

0.8614401496259352

In [154]:
#window_and_purchase.to_csv('window_and_purchase.csv')